# Pokemon Winner Classification


## Setup

In [1]:
# some useful mysklearn package import statements and reloads
import importlib
import os

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [2]:
filename = os.path.join("input_data", "pokemon_combats.csv")
table = MyPyTable().load_from_file(filename)

# pokemon type
type_1_first = table.get_column("Type 1_first")
type_2_first = table.get_column("Type 2_first")
type_1_second = table.get_column("Type 1_second")
type_2_second = table.get_column("Type 2_second")
X_types = [[t1f, t2f, t1s, t2s] for t1f, t2f, t1s, t2s in zip(type_1_first, type_2_first, type_1_second, type_2_second)]
y = table.get_column("Winner")
train_sets_types, test_sets_types = myevaluation.stratified_kfold_cross_validation(X_types, y, n_splits=10, random_state=0, shuffle=False)

# # game statistics
# field_goals = table.get_column("RegularSeasonFGPercentMean")
# three_ptrs = table.get_column("RegularSeasonFG3PercentMean")
# turn_overs = table.get_column("RegularSeasonTOMean")
# steals = table.get_column("RegularSeasonStlMean")
# X_stats = [[fg, tp, to, s] for fg, tp, to, s in zip(field_goals, three_ptrs, turn_overs, steals)]
# train_sets_stats, test_sets_stats = myevaluation.stratified_kfold_cross_validation(X_stats, y, n_splits=10, random_state=0, shuffle=True)

## Predicting Winners Using Pokemon Type
Pokemon is essentially more complicated game of rock, paper, scissors in that some pokemon have type advantages that make them strong or resistant against other types. For instance, Water types are strong against Fire types, but weak against Grass types. Thus, the best predictor of who won a battle would likely be Pokemon typings. Each Pokemon can have up to two different types of the following types: Poison, Flying, Dragon, Ground, Fairy, Grass, Fighting, Psychic, Steel, Ice, Rock, Dark, Water, Electric, Fire, Ghost, Bug, Normal.

To test how well typing serves as a predictor, kNN, Naive Bayes, Dummy, and Decision Tree classifiers made predictions for each test set in each fold. All predictions are stored for performance scoring against `y_true`, the correct classifier for each test instance. Several metrics for performance were compared among the classifiers. 

In [3]:
knn = MyKNeighborsClassifier(n_neighbors=3)
nb = MyNaiveBayesClassifier()
dummy = MyDummyClassifier()
dt = MyDecisionTreeClassifier()

# accuracy is the total correctly predicted divided by the total predicted over all the folds
knn_correct, nb_correct, dummy_correct, dt_correct, total_predicted = 0, 0, 0, 0, 0
knn_preds, nb_preds, dummy_preds, dt_preds, y_true = [], [], [], [], []

for train, test in zip(train_sets_types, test_sets_types):
    X_train = [X_types[i] for i in train]
    y_train = [y[i] for i in train]
    X_test = [X_types[i] for i in test]
    y_test = [y[i] for i in test]
    total_predicted += len(y_test)
    y_true += y_test

    # kNN
    # print(X_train)
    knn.fit(X_train, y_train)
    knn_pred = knn.predict(X_test)
    knn_preds += knn_pred
    knn_correct += myevaluation.accuracy_score(y_test, knn_pred, normalize=False)

    # Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_preds += nb_pred
    nb_correct += myevaluation.accuracy_score(y_test, nb_pred, normalize=False)

    # Dummy
    dummy.fit(X_train, y_train)
    dummy_pred = dummy.predict(X_test)
    dummy_preds += dummy_pred
    dummy_correct += myevaluation.accuracy_score(y_test, dummy_pred, normalize=False)

    # Decision Tree
    dt.fit(X_train, y_train)
    dt_pred = dt.predict(X_test)
    dt_preds += dt_pred
    dt_correct += myevaluation.accuracy_score(y_test, dt_preds, normalize=False)

## Classifier Performance Using Pokemon Type
How well can typing predict the winning Pokemon?

### kNN
kNN had a relatively low accuracy for a binary class prediction with 48.2% accuracy. kNN seemed to predict "A" almost every time, leading to an extremely low recall score of 7%.

In [ ]:
labels = [1, 2]
pos_label = 1

knn_accuracy = knn_correct / total_predicted
knn_precision = myevaluation.binary_precision_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_recall = myevaluation.binary_recall_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_f1 = myevaluation.binary_f1_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_confusion_matrix = myevaluation.confusion_matrix(y_true, knn_preds, labels)

myutils.print_results("kNN Classifier", knn_accuracy, knn_precision, knn_recall, knn_f1)
header = ["Winner", "H", "A"]
myutils.print_confusion_matrix(header, knn_confusion_matrix, labels)


kNN Classifier
accuracy score: 0.482
error rate: 0.518
precision score: 0.542
recall score: 0.074
f1 score: 0.131

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H          13  162      175              7.429
A          11  148      159             93.082


### Naive Bayes
Naive Bayes performed the best of the classifiers with an accuracy of 68.9%. Precision, recall, and f1 scores were also high (>68%).

In [ ]:
nb_accuracy = nb_correct / total_predicted
nb_precision = myevaluation.binary_precision_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_recall = myevaluation.binary_recall_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_f1 = myevaluation.binary_f1_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_confusion_matrix = myevaluation.confusion_matrix(y_true, nb_preds, labels)

myutils.print_results("Naive Bayes Classifier", nb_accuracy, nb_precision, nb_recall, nb_f1)
myutils.print_confusion_matrix(header, nb_confusion_matrix, labels)


Naive Bayes Classifier
accuracy score: 0.689
error rate: 0.311
precision score: 0.682
recall score: 0.76
f1 score: 0.719

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H         133   42      175             76
A          62   97      159             61.006


### Dummy
Dummy performed with 47.3% accuracy, same as kNN. Because Dummy uses the majority class label and classifications were binary, Dummy had an excellent recall score of 85.7%, but a terribly low precision score of ~5%.

In [ ]:
dummy_accuracy = dummy_correct / total_predicted
dummy_precision = myevaluation.binary_precision_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_recall = myevaluation.binary_recall_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_f1 = myevaluation.binary_f1_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_confusion_matrix = myevaluation.confusion_matrix(y_true, dummy_preds, labels)

myutils.print_results("Dummy Classifier", dummy_accuracy, dummy_precision, dummy_recall, dummy_f1)
myutils.print_confusion_matrix(header, dummy_confusion_matrix, labels)


Dummy Classifier
accuracy score: 0.473
error rate: 0.527
precision score: 0.498
recall score: 0.857
f1 score: 0.63

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H         150   25      175             85.714
A         151    8      159              5.031


### Decision Tree
Decision tree performed with 52.4% accuracy rate and relatively low precision, recall, and f1 scores (~32%).

In [ ]:
dt_accuracy = dt_correct / total_predicted
dt_precision = myevaluation.binary_precision_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_recall = myevaluation.binary_recall_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_f1 = myevaluation.binary_f1_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_confusion_matrix = myevaluation.confusion_matrix(y_true, dt_preds, labels)

myutils.print_results("Decision Tree Classifier", dt_accuracy, dt_precision, dt_recall, dt_f1)
myutils.print_confusion_matrix(header, dt_confusion_matrix, labels)


Decision Tree Classifier
accuracy score: 0.524
error rate: 0.476
precision score: 0.337
recall score: 0.314
f1 score: 0.325

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H          55  120      175             31.429
A         108   51      159             32.075


## Predicting Winner Using Pokemon Stats
kNN, Naive Bayes, Dummy, and Decision Tree classifiers make predictions for each test set in each fold. All predictions are stored for performance scoring against `y_true`, the correct classifier for each test instance. Attributes used for predictions are:

In [ ]:
# accuracy is the total correctly predicted divided by the total predicted over all the folds
knn_correct, nb_correct, dummy_correct, dt_correct, total_predicted = 0, 0, 0, 0, 0
knn_preds, nb_preds, dummy_preds, dt_preds, y_true = [], [], [], [], []

for train, test in zip(train_sets_stats, test_sets_stats):
    X_train = [X_stats[i] for i in train]
    y_train = [y_stats[i] for i in train]
    X_test = [X_stats[i] for i in test]
    y_test = [y_stats[i] for i in test]
    total_predicted += len(y_test)
    y_true += y_test

    # kNN
    # print(X_train)
    knn.fit(X_train, y_train)
    knn_pred = knn.predict(X_test)
    knn_preds += knn_pred
    knn_correct += myevaluation.accuracy_score(y_test, knn_pred, normalize=False)

    # Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_preds += nb_pred
    nb_correct += myevaluation.accuracy_score(y_test, nb_pred, normalize=False)

    # Dummy
    dummy.fit(X_train, y_train)
    dummy_pred = dummy.predict(X_test)
    dummy_preds += dummy_pred
    dummy_correct += myevaluation.accuracy_score(y_test, dummy_pred, normalize=False)

    # Decision Tree
    dt.fit(X_train, y_train)
    dt_pred = dt.predict(X_test)
    dt_preds += dt_pred
    dt_correct += myevaluation.accuracy_score(y_test, dt_preds, normalize=False)

## Classifier Performance Using Pokemon Statistics

### kNN
kNN scored an accuracy of 53% and slightly lower precision, recall, and f1 scores.

In [ ]:
labels = ["H", "A"]
pos_label = "H"

knn_accuracy = knn_correct / total_predicted
knn_precision = myevaluation.binary_precision_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_recall = myevaluation.binary_recall_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_f1 = myevaluation.binary_f1_score(y_true, knn_preds, labels=labels, pos_label=pos_label)
knn_confusion_matrix = myevaluation.confusion_matrix(y_true, knn_preds, labels)

myutils.print_results("kNN Classifier", knn_accuracy, knn_precision, knn_recall, knn_f1)
header = ["Winner", "H", "A"]
myutils.print_confusion_matrix(header, knn_confusion_matrix, labels)


kNN Classifier
accuracy score: 0.53
error rate: 0.47
precision score: 0.567
recall score: 0.434
f1 score: 0.492

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H          76   99      175             43.429
A          58  101      159             63.522


### Naive Bayes
Once again, Naive Bayes performed the best of the classifiers with an accuracy of 61%. Precision, recall, and f1 were all also in the range of 60-65%.

In [ ]:
nb_accuracy = nb_correct / total_predicted
nb_precision = myevaluation.binary_precision_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_recall = myevaluation.binary_recall_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_f1 = myevaluation.binary_f1_score(y_true, nb_preds, labels=labels, pos_label=pos_label)
nb_confusion_matrix = myevaluation.confusion_matrix(y_true, nb_preds, labels)

myutils.print_results("Naive Bayes Classifier", nb_accuracy, nb_precision, nb_recall, nb_f1)
myutils.print_confusion_matrix(header, nb_confusion_matrix, labels)


Naive Bayes Classifier
accuracy score: 0.587
error rate: 0.413
precision score: 0.603
recall score: 0.617
f1 score: 0.61

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H         108   67      175             61.714
A          71   88      159             55.346


### Dummy
Dummy performed with an accuracy of 47.3%. Due to the mechanics of Dummy, all performance stats are the same as above.

In [ ]:
dummy_accuracy = dummy_correct / total_predicted
dummy_precision = myevaluation.binary_precision_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_recall = myevaluation.binary_recall_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_f1 = myevaluation.binary_f1_score(y_true, dummy_preds, labels=labels, pos_label=pos_label)
dummy_confusion_matrix = myevaluation.confusion_matrix(y_true, dummy_preds, labels)

myutils.print_results("Dummy Classifier", dummy_accuracy, dummy_precision, dummy_recall, dummy_f1)
myutils.print_confusion_matrix(header, dummy_confusion_matrix, labels)


Dummy Classifier
accuracy score: 0.473
error rate: 0.527
precision score: 0.498
recall score: 0.857
f1 score: 0.63

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H         150   25      175             85.714
A         151    8      159              5.031


### Decision Tree
Decision tree surprisingly performed the worst, with an accuracy of 45.8%. This is likely due to overfitting. Pruning the decision tree could help performance (see below for pruning analysis).

In [ ]:
dt_accuracy = dt_correct / total_predicted
dt_precision = myevaluation.binary_precision_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_recall = myevaluation.binary_recall_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_f1 = myevaluation.binary_f1_score(y_true, dt_preds, labels=labels, pos_label=pos_label)
dt_confusion_matrix = myevaluation.confusion_matrix(y_true, dt_preds, labels)

myutils.print_results("Decision Tree Classifier", dt_accuracy, dt_precision, dt_recall, dt_f1)
myutils.print_confusion_matrix(header, dt_confusion_matrix, labels)


Decision Tree Classifier
accuracy score: 0.458
error rate: 0.542
precision score: 0.49
recall score: 0.44
f1 score: 0.464

Winner      H    A    Total    Recognition (%)
--------  ---  ---  -------  -----------------
H          77   98      175             44
A          80   79      159             49.686
